### Load Diagnoses

In [33]:
import pandas as pd

# persons with 3 or more HIV dx codes after 2018-01-01
cohort = pd.read_csv("HIV_cohort.csv")

# target conditions like HCV/obesity/depression/anxiety/etc.
conditions = pd.get_dummies(pd.read_csv("condition_query.csv"), 
                           prefix=None).groupby('person_id').sum().reset_index()

# merge aforementioned dataframes
dx_df = cohort.merge(conditions,
            how = 'left', on = 'person_id').fillna(0)

### Outpatient Visits, Emergency Deparment Encounters, and Inpatient Admissions

In [34]:
# number of E&M visits for established patients (CPT 99211-99215) from 6-2018 to 6-2019
outpatient = pd.read_csv("outpatient_visits.csv")
outpatient['office_visits'] = outpatient['visits']

# number of ED and inpatient encounters from 6-2018 to 6-2019
hospital = pd.read_csv("hospital_visits.csv")

visits_df = outpatient[['person_id', 'office_visits']].merge(hospital, 
                on = 'person_id', how = 'outer').fillna(0)

### Laboratory Values: HIV Viral Load and HbA1c

In [35]:
######################
# HIV RNA viral load #
######################

# all viral load labs starting 2018-01-01
vl_labs = pd.read_csv("viral_loads.csv")

# convert strings to float using a 'reference data frame'
vl_summary = vl_labs.groupby('value_source_value').size().reset_index().sort_values([0], ascending = False)
vl_summary.columns = ['value_source_value', 'value']

# normalize VL labs
i, floats, dmap = 0, [], {'Not Detected':0, '<20':0, '>10000000':10000, 'BT':0, 'TNP':0}
for result in vl_summary['value_source_value'].tolist():
    try:
        floats.append(float(result))
    except:
        floats.append(float(dmap[result]))

# vls column to create binary variable for 'viral load suppression'
vl_summary['vls'] = [x < 200 for x in floats]

VLS_df = vl_labs.merge(vl_summary, 
                       on = 'value_source_value')[['person_id', 'measurement_date', 'vls']]

# get most recent lab for each patient
VLS_df = VLS_df.merge(
    VLS_df.groupby('person_id').measurement_date.first().reset_index(),
    on = ['person_id', 'measurement_date'])



#########
# HbA1c #
#########

hba1c_df = pd.read_csv('hba1c_values.csv').sort_values(['person_id','measurement_date'])
# get most recent lab for each patient
hba1c_df = hba1c_df.groupby('person_id').last()
hba1c_df['hba1c'] = hba1c_df['value_as_number']

#############
# join labs #
#############

lab_df = VLS_df.merge(hba1c_df, on = 'person_id', how = 'left')

### Create Final Dataframe

In [39]:
df = dx_df.merge(visits_df, how = 'left').fillna(0).merge(
    lab_df[['person_id', 'vls', 'hba1c']], on = 'person_id')

#df.to_csv("omop_indicators.csv")

,person_id,dx_alcoholism,dx_anxiety,dx_depression,dx_diabetes,dx_drug abuse,dx_hcv,dx_hypertension,dx_psychosis,office_visits,er_visits,inpatient_admissions,vls,hba1c
0,882726,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,True,4.7
1,2801900,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,NaN
2,2004484,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,6.0,1.0,0.0,True,9.6
3,1905426,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,9.2
4,3867637,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,5.3
5,1928025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,True,4.4
6,1934623,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,5.5
7,2234562,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,2.0,2.0,True,4.9
8,3183880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,True,4.6
9,9520883,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,True,4.9


In [26]:
df.columns
# need new diagnosis, hospitalization risk, CD4 count, unstable housing

Index(['person_id', 'visits', 'dx_alcoholism', 'dx_anxiety', 'dx_depression',
       'dx_diabetes', 'dx_drug abuse', 'dx_hcv', 'dx_hypertension',
       'dx_psychosis', 'vls', 'hba1c'],
      dtype='object')